# LLM Evaluation - Perplexity
Using Perplexity with Gemma-2-27B and TildeOpen-30B

In [ ]:
!pip install transformers requests torch
!pip install sentencepiece --no-deps --force-reinstall

In [ ]:
import requests
import math
from transformers import AutoTokenizer

ENDPOINTS = {
    "google/gemma-2-27b": "http://gauja.ailab.lv:30052/v1",
    "TildeAI/TildeOpen-30b": "http://gauja.ailab.lv:30053/v1"
}

In [ ]:
def query_model(model_name, text):
    url = ENDPOINTS[model_name] + "/completions"
    payload = {
        "model": model_name,
        "prompt": text,
        "max_tokens": 1, # disables text continuation
        "temperature": 0.0,
        "logprobs": True,
        "prompt_logprobs": True # required for perplexity computation
    }
    r = requests.post(url, json=payload)
    r.raise_for_status()
    return r.json()

In [ ]:
def extract_prompt_logprobs(response):
    prompt_lp = response["choices"][0]["prompt_logprobs"]
    lps = []

    for entry in prompt_lp:
        if entry is None:
            # this is the BOS token, skip it
            continue

        # entry is a dict: {token_id: {"decoded_token": ..., "logprob": ..., "rank": ...}, ...}
        # we want the gold token (rank == 1) or the minimum rank
        gold = min(entry.values(), key=lambda x: x["rank"])
        lps.append(gold["logprob"])

    return lps

In [ ]:
def token_ppl(logprobs):
    return math.exp(-sum(logprobs) / len(logprobs))

def char_ppl(logprobs, text):
    return math.exp(-sum(logprobs) / len(text))

In [ ]:
def compare_models(text):
    results = {}
    for model in ENDPOINTS:
        resp = query_model(model, text)
        lps = extract_prompt_logprobs(resp)
        results[model]=char_ppl(lps, text)
    return results

In [ ]:
# model comparison using char perplexity
texts = {'Latvian': 'Sētniece paķēra nazi un kurvi un rāvās sēnei pakaļ. Panāca aiz nākamā ielu krustojuma.\n\n    Tad viņa attapās, ka nav taču jāskrien: sēnes nāca un nāca pa ielu šurp, pašas pretī, un sētniece apsēdās ielas malā uz trotuāra, nolika kurvi blakus un — kā tikai nāk sēne, tā ķer ciet un grozā iekšā. Kas tie bija par priekiem! Drīz visa iela bija pilna ar sēņotājiem, bet mežs plūda un plūda garām kā zaļa upe, un sēņotāji ielas malās izskatījās kā makšķerētāji upmalā.\n\n   Pa citām ielām gāja lazdas, zaļos zarus gar namu logiem braucīdamas, un riekstus varēja raut tieši no loga, ja kāds bija tik slinks, ka nevīžoja iziet uz ielas. ',
         'Latgalian': 'Senējūs laikūs beja krūgs. Uz krūgu staiguoja cīmu meitas doncuot. Kaidā vokorā tuos meitas, kuras jau daudz reižu beja doncuojušas krūgā, aizvedja jaunu meitiņu, kas vēl ni reizes nabeja bejuse krūgā. Kotrai meitai beja sovs bryutguons, ari jaunajai meitiņai atsaroda sovs.\n\nVokorā doncuojūt jaunuo meitiņa īsavēŗa sovam bryutguonam asti, un nu tik tūlaik sapr ota, ka vysi bryutguoni ir valni. Meitiņa izzamuonīja uorā nu laidjās bēgt. Skrēja, skrēja jei nu pīsalīcja pi zemes pasaklausīt, vai kas nasadzan pakaļ. Zeme vīn reibēja, kai valns dzynuos pakaļ.\n\nMeitiņa beja nūguruse un īskrēja rejā, kas beja pīsārta ar lynim, izkuopja uz lynim un sēd. Atskrēja valns un soka: “Kur te kristīta cylvāka smoka?”\n\nLyni žāluoja meitiņas un suoka stuostīt garu stuostu par sovu dzeivi, kai jī patyka te kula rejā. Valns grib, ka lyni dreižuok izstuostītu, bet lyni vēl lēnuok stuostīja.',
         'Greek': 'Χρησιμοποιούμε cookies που είναι αναγκαία για την ορθή λειτουργία του ιστότοπου. Με τη συγκατάθεσή σας θα χρησιμοποιήσουμε και cookies για να συλλέξουμε στατιστικά στοιχεία σχετικά με την επισκεψιμότητα στην Ενιαία Ψηφιακή Πύλη gov.gr με σκοπό τη βελτίωση της επίδοσης και του περιεχομένου της. Μπορείτε να ενημερωθείτε αναλυτικά για την Πολιτική Προστασίας Προσωπικών Δεδομένων',
         'English': "At least 25 people have been killed including three children in a Russian drone and missile attack on the western city of Ternopil that hit two blocks of flats, Ukrainian rescue officials say.\n\nAnother 73 people were wounded, 15 of them children, officials said, in one of the deadliest Russian strikes on western Ukraine since Moscow launched a full-scale war in 2022.\n\nTwo other western regions were hit, Lviv and Ivano-Frankivsk, and a drone attack targeted three districts of the northern city of Kharkiv, wounding more than 30 people. Photos posted online showed buildings and cars ablaze.\n\nUkraine's air force said it had shot down 442 of 476 drones launched by Russia and 41 of 48 missiles that were fired.",
         'Finnish': "Suomi on päättänyt hyödyntää EU:n SAFE-rahoitusvälinettä miljardilla eurolla puolustusteollisuuden vahvistamiseksi. Rahoituksen käyttöä suunnitellaan Suomen maataistelukykyä vahvistaviin monikansallisiin yhteishankintoihin ja erityisesti droonisuorituskykyjen kehittämiseen yhteistyössä Ukrainan ja liittolaisten kanssa. Hallituksen talouspoliittinen ministerivaliokunta linjasi asiasta tiistaina 18. marraskuuta. \n\n”Päätöksellä vahvistamme Suomen puolustusteollista kapasiteettia, syvennämme eurooppalaista puolustusyhteistyötä sekä luomme kasvumahdollisuuksia kotimaisille yrityksille”, sanoo pääministeri Petteri Orpo. \n\n”Tämä on tärkeä päätös, joka vahvistaa Suomen puolustusta, suomalaisen puolustusteollisuuden edellytyksiä sekä eurooppalaista puolustusyhteistyötä ja yhteishankintoja”, sanoo puolustusministeri Antti Häkkänen.\nSAFE-välineen ensisijaiset hyödyt ovat suomalaisen puolustusteollisuuden tuotteiden tilausten kasvattamisessa kumppanimaiden kanssa ja suomalaisten hankkeiden strategisena tunnustamisena EU:n puolustusteollisuuspolitiikassa. Välineellä tuetaan myös Ukrainan puolustusteollisuutta ja edistetään sen integraatiota osaksi eurooppalaista puolustusyhteistyötä.",
         'Russian': "Мирный план США по Украине может быть согласован всеми заинтересованными сторонами уже к концу ноября или даже «на этой неделе», сообщил Politico высокопоставленный представитель Белого дома, отметивший, что администрация находится на «грани серьезного прорыва». Об этом стало известно после сообщения Axios о том, что команда президента Дональда Трампа вместе с РФ тайно работает над новым планом по прекращению войны в Украине. Состоящий из 28 пунктов план, как утверждали собеседники издания, включает четыре направления: мир в Украине, гарантии безопасности, безопасность в Европе и будущие отношения США с Россией и Украиной.\n\nОднако в составлении нового «рамочного» мирного соглашения, вероятно, напрямую не участвовали ни Киев, ни союзники США в Европе, отмечает издание. Американский чиновник сообщил, что в Белом доме настроены оптимистично и собираются презентовать свой план президенту Владимиру Зеленскому как «свершившийся факт». «То, что мы собираемся представить [Украине], — разумно», — заявил высокопоставленный представитель Белого дома."
        }

for lang, text in texts.items():
    print(lang, compare_models(text))

Latvian {'google/gemma-2-27b': 1.4734850560301704, 'TildeAI/TildeOpen-30b': 1.329736557118415}
Latgalian {'google/gemma-2-27b': 1.796697701976164, 'TildeAI/TildeOpen-30b': 1.2495486717499509}
Greek {'google/gemma-2-27b': 1.1504216103639386, 'TildeAI/TildeOpen-30b': 1.1876752585152566}
English {'google/gemma-2-27b': 1.1844912025823093, 'TildeAI/TildeOpen-30b': 1.1992852188187402}
Finnish {'google/gemma-2-27b': 1.1813382268311305, 'TildeAI/TildeOpen-30b': 1.150873657239602}
Russian {'google/gemma-2-27b': 1.2069246317865088, 'TildeAI/TildeOpen-30b': 1.193389823624944}


In [ ]:
def evaluate_model(model, text):
    resp = query_model(model, text)
    lps = extract_prompt_logprobs(resp)
    return token_ppl(lps)

In [ ]:
#MultiBLiMP evaluating a model using contrastive pairs
contrastive_pairs= {
    'latvian': ["Anna un Pēteris ir mācījies Rīgas Valsts 1. ģimnāzijā.", "Anna un Pēteris ir mācījušies Rīgas Valsts 1. ģimnāzijā."],
    'english': ["Artūrs Vasiļevskis and me are privileged to be invited to represent Tilde, and perhaps more broadly, the Eastern flank of the European continent at the inaugural workshop on the Pretraining and Post-training of Open Source Sovereign Foundation Models.", "Artūrs Vasiļevskis and I are privileged to be invited to represent Tilde, and perhaps more broadly, the Eastern flank of the European continent at the inaugural workshop on the Pretraining and Post-training of Open Source Sovereign Foundation Models."]
}

for lang, texts in contrastive_pairs.items():
    print(lang)
    for text in texts:
        print(evaluate_model("TildeAI/TildeOpen-30b", text), text)

latvian
2.5907057017481963 Anna un Pēteris ir mācījies Rīgas Valsts 1. ģimnāzijā.
2.3017945054726328 Anna un Pēteris ir mācījušies Rīgas Valsts 1. ģimnāzijā.
english
2.696991410014835 Artūrs Vasiļevskis and me are privileged to be invited to represent Tilde, and perhaps more broadly, the Eastern flank of the European continent at the inaugural workshop on the Pretraining and Post-training of Open Source Sovereign Foundation Models.
2.6059035197385514 Artūrs Vasiļevskis and I are privileged to be invited to represent Tilde, and perhaps more broadly, the Eastern flank of the European continent at the inaugural workshop on the Pretraining and Post-training of Open Source Sovereign Foundation Models.


In [ ]:
#Exams and multiple choice questions
# Suppose you have a test question:
"""Kolektivizācijas sekas bija:
A: Pamazām iznīka lauku viensētas.
B: Iedzīvotāji ienākumu meklējumos pārcēlās uz dzīvi pilsētās.
C: Daudzi lauku iedzīvotāji zaudēja darbu.
D: Radās daudz salīdzinoši nelielu zemnieku saimniecību.
Pareizā Atbilde: A: Pamazām iznīka lauku viensētas."""
# Use ideas from above and the lecture to test either of the models

'Kolektivizācijas sekas bija:\nA: Pamazām iznīka lauku viensētas.\nB: Iedzīvotāji ienākumu meklējumos pārcēlās uz dzīvi pilsētās.\t\nC: Daudzi lauku iedzīvotāji zaudēja darbu.\nD: Radās daudz salīdzinoši nelielu zemnieku saimniecību.\nPareizā Atbilde: A: Pamazām iznīka lauku viensētas.'